In [1]:
import os
import medusa
from pickle import load
import pandas as pd
from medusa.flux_analysis.deletion import ensemble_single_reaction_deletion
from medusa.flux_analysis.deletion import ensemble_single_gene_deletion

import matplotlib.pyplot as plt

/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# load all ensembles
ensembles = {}
exclude = ['Bacillus megaterium']
for species_file in os.listdir("../results/ensembles/"):
    species_name = species_file.split(".")[0]
    
    if species_name not in exclude:
        with open("../results/ensembles/"+species_name+".pickle",'rb') as infile:
            ensembles[species_name] = load(infile)

In [ ]:
reaction_knockout_results = {}
for species in ensembles.keys():
    ensemble = ensembles[species]
    print("Starting knockouts for " + species)
    # make sure all exchange reactions are open to calculate
    # growth rate in complete media
    ex_rxns = [rxn for rxn in ensemble.base_model.reactions if
                  rxn.id.startswith('EX_')]
    for rxn in ex_rxns:
        rxn.lower_bound = -1000
        
    reaction_knockout_results[species] = ensemble_single_reaction_deletion(ensemble)
    species_df = pd.DataFrame()
    for member in reaction_knockout_results:
        reaction_knockout_results[member].index = [
            list(reaction)[0] for reaction in reaction_knockout_results[member].index]
        species_df[member] = reaction_knockout_results[member]['growth']
    species_df.to_csv('../results/reaction_knockouts/'+species+'.csv',sep=',')

Starting knockouts for Ralstonia pickettii
performing deletions for Ralstonia pickettii_gapfilled_61
performing deletions for Ralstonia pickettii_gapfilled_65
performing deletions for Ralstonia pickettii_gapfilled_7
performing deletions for Ralstonia pickettii_gapfilled_46
performing deletions for Ralstonia pickettii_gapfilled_97
performing deletions for Ralstonia pickettii_gapfilled_59
performing deletions for Ralstonia pickettii_gapfilled_70
performing deletions for Ralstonia pickettii_gapfilled_47
performing deletions for Ralstonia pickettii_gapfilled_54
performing deletions for Ralstonia pickettii_gapfilled_85
performing deletions for Ralstonia pickettii_gapfilled_25
performing deletions for Ralstonia pickettii_gapfilled_34
performing deletions for Ralstonia pickettii_gapfilled_50
performing deletions for Ralstonia pickettii_gapfilled_26
performing deletions for Ralstonia pickettii_gapfilled_31
performing deletions for Ralstonia pickettii_gapfilled_93
performing deletions for Ralst

In [ ]:
gene_knockout_results = {}
for species in ensembles.keys():
    if not species == 'Pseudomonas mendocina':
        ensemble = ensembles[species]
        print("Starting knockouts for " + species)
        # make sure all exchange reactions are open to calculate
        # growth rate in complete media
        ex_rxns = [rxn for rxn in ensemble.base_model.reactions if
                      rxn.id.startswith('EX_')]
        for rxn in ex_rxns:
            rxn.lower_bound = -1000

        gene_knockout_results = ensemble_single_gene_deletion(ensemble)
        species_df = pd.DataFrame()
        for member in gene_knockout_results:
            gene_knockout_results[member].index = [list(gene)[0] for gene in gene_knockout_results[member].index]
            species_df[member] = gene_knockout_results[member]['growth']
        species_df.to_csv('../results/gene_knockouts/'+species+'.csv',sep=',')